In [11]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 23: Category Six ---</h2><p>The droids have finished repairing as much of the ship as they can.  Their <span title="If you don't at least exhale sharply through your nose after reading this joke, we probably can't be friends.">report</span> indicates that this was a <em>Category 6</em> disaster - not because it was that bad, but because it destroyed the stockpile of <a href="https://en.wikipedia.org/wiki/Category_6_cable">Category 6</a> network cables as well as most of the ship's network infrastructure.</p>
<p>You'll need to <em>rebuild the network from scratch</em>.</p>
<p>The computers on the network are standard <a href="9">Intcode</a> computers that communicate by sending <em>packets</em> to each other.  There are <code>50</code> of them in total, each running a copy of the same <em>Network Interface Controller</em> (NIC) software (your puzzle input). The computers have <em>network addresses</em> <code>0</code> through <code>49</code>; when each computer boots up, it will request its network address via a single input instruction. Be sure to give each computer a unique network address.</p>
<p>Once a computer has received its network address, it will begin doing work and communicating over the network by sending and receiving <em>packets</em>. All packets contain <em>two values</em> named <code>X</code> and <code>Y</code>. Packets sent to a computer are queued by the recipient and read in the order they are received.</p>
<p>To <em>send</em> a packet to another computer, the NIC will use <em>three output instructions</em> that provide the <em>destination address</em> of the packet followed by its <code>X</code> and <code>Y</code> values.  For example, three output instructions that provide the values <code>10</code>, <code>20</code>, <code>30</code> would send a packet with <code>X=20</code> and <code>Y=30</code> to the computer with address <code>10</code>.</p>
<p>To <em>receive</em> a packet from another computer, the NIC will use an <em>input instruction</em>.  If the incoming packet queue is <em>empty</em>, provide <code>-1</code>.  Otherwise, provide the <code>X</code> value of the next packet; the computer will then use a second input instruction to receive the <code>Y</code> value for the same packet.  Once both values of the packet are read in this way, the packet is removed from the queue.</p>
<p>Note that these input and output instructions never <a href="https://en.wikipedia.org/wiki/Blocking_(computing)">block</a>. Specifically, output instructions do not wait for the sent packet to be received - the computer might send multiple packets before receiving any. Similarly, input instructions do not wait for a packet to arrive - if no packet is waiting, input instructions should receive <code>-1</code>.</p>
<p>Boot up all <code>50</code> computers and attach them to your network.  <em>What is the <code>Y</code> value of the first packet sent to address <code>255</code>?</em></p>
</article>


In [12]:
with open("../input/day23.txt") as f:
    puzzle = f.read()

In [13]:
from collections import deque
from copy import deepcopy

from more_itertools import last
from IntcodeComputer import IntcodeComputer


class NetWork:
    def __init__(self, program: str) -> None:
        nic = IntcodeComputer(program)

        self.nic_states = [deepcopy(nic) for _ in range(50)]
        self.nics = [deepcopy(nic).run() for ns in self.nic_states]
        self.message_queue = [deque([]) for _ in range(50)]

    def boot(self) -> None:
        for network_adress, nic in enumerate(self.nics):
            nic.send(None)
            nic.send(network_adress)

    def networking(self) -> int:
        while True:
            for i, (state, nic, mq) in enumerate(
                zip(self.nic_states, self.nics, self.message_queue)
            ):
                if not mq:
                    other_nic = nic.send(-1)
                else:
                    x, y = mq.popleft()
                    nic.send(x)
                    other_nic = nic.send(y)

                while other_nic is not None and not state.waiting_for_input:
                    x = nic.send(None)
                    y = nic.send(None)

                    if other_nic == 255:
                        return x, y

                    mq_to = self.message_queue[other_nic]
                    mq_to.append((x, y))
                    if not state.waiting_for_input:
                        other_nic = nic.send(y)

    def networking_with_NAT(self) -> int:
        NAT_last_package = None, None
        NAT_last_y_send = None

        while True:
            active = False

            for i, (state, nic, mq) in enumerate(
                zip(self.nic_states, self.nics, self.message_queue)
            ):
                if not mq:
                    other_nic = nic.send(-1)
                else:
                    x, y = mq.popleft()
                    nic.send(x)
                    other_nic = nic.send(y)

                while other_nic is not None and not state.waiting_for_input:
                    active = True
                    x = nic.send(None)
                    y = nic.send(None)

                    if other_nic == 255:
                        NAT_last_package = x, y
                    else:
                        mq_to = self.message_queue[other_nic]
                        mq_to.append((x, y))
                    if not state.waiting_for_input:
                        other_nic = nic.send(None)

            if not active and NAT_last_package != (None, None):
                x, y = NAT_last_package

                if y == NAT_last_y_send:
                    return y

                NAT_last_y_send = y

                self.message_queue[0].append((x, y))


nw = NetWork(puzzle)
nw.boot()
print(f"Part I: {last(nw.networking())}")

Part I: 15662


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>15662</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Packets sent to address <code>255</code> are handled by a device called a NAT (Not Always Transmitting). The NAT is responsible for managing power consumption of the network by blocking certain packets and watching for idle periods in the computers.</p>
<p>If a packet would be sent to address <code>255</code>, the NAT receives it instead.  The NAT remembers only the <em>last</em> packet it receives; that is, the data in each packet it receives overwrites the NAT's packet memory with the new packet's <code>X</code> and <code>Y</code> values.</p>
<p>The NAT also monitors all computers on the network.  If all computers have <em>empty incoming packet queues</em> and are <em>continuously trying to receive packets</em> without sending packets, the network is considered <em>idle</em>.</p>
<p>Once the network is idle, the NAT sends <em>only the last packet it received</em> to address <code>0</code>; this will cause the computers on the network to resume activity.  In this way, the NAT can throttle power consumption of the network when the ship needs power in other areas.</p>
<p>Monitor packets released to the computer at address <code>0</code> by the NAT.  <em>What is the first <code>Y</code> value delivered by the NAT to the computer at address <code>0</code> twice in a row?</em></p>
</article>

</main>


In [14]:
nw = NetWork(puzzle)
nw.boot()
print(f"Part II: {nw.networking_with_NAT()}")

Part II: 10854


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>10854</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
